In [1]:
import pandas as pd
import numpy as np

In [2]:
df_sat = pd.read_csv("../../data/park-movement-Sat.csv")
df_fri = pd.read_csv("../../data/park-movement-Fri-FIXED-2.0.csv")
df_sun = pd.read_csv("../../data/park-movement-Sun.csv")
checkins = pd.read_csv("../../data/checkin_locations.csv")

In [3]:
df = pd.concat([df_fri, df_sat, df_sun])

In [4]:
id_data = df_sun

In [5]:
#id_data = id_data.drop_duplicates(['X', 'Y'])
#d_data = id_data.reset_index(drop=True)

In [6]:
import json 
with open('../../data/pop_count_day_hour/regMapping.json', 'r') as f:
    regMap = json.load(f)

In [7]:
rMap = {}
rmap = regMap['mapping']
for json in rmap:
    for key in json:
        rMap[key] = json[key]        

In [8]:
def assignRegions(df):
    rmap = rMap
    x = df['X']
    y = df['Y']
    #print(rmap)
    for key in rmap:
        if x < rmap[key]['low_x'] or x > rmap[key]['high_x']:
#             if(key=='entryCorridor'):
#                 print(x, rmap[key]['low_x'],rmap[key]['high_x'])
            continue
        if  y < rmap[key]['low_y'] or y > rmap[key]['high_y']:
#             if(key=='entryCorridor'):
#                 print(y, rmap[key]['low_y'],rmap[key]['high_y'])
            continue
        ## Yeah point inside this region
        return key
    return 'CoasterAlley'
            

In [9]:
rmap

[{'wetLand': {'low_x': 0, 'low_y': 34, 'high_x': 82.99, 'high_y': 53.99}},
 {'tunda': {'low_x': 0, 'low_y': 54, 'high_x': 50.99, 'high_y': 99}},
 {'entryCorridor': {'low_x': 51, 'low_y': 54, 'high_x': 69.99, 'high_y': 99}},
 {'kiddleLand': {'low_x': 83, 'low_y': 54, 'high_x': 99, 'high_y': 99}}]

In [ ]:
id_data['region'] = id_data[['X','Y']].apply(lambda x: assignRegions(x), axis=1)

In [ ]:
id_data.head()

In [ ]:
num_checkins = id_data[['id','region']].groupby('region').count()

In [ ]:
num_checkins

In [ ]:
final_data = pd.merge(left=id_data, right=checkins, how='left', on=['X','Y'])

In [ ]:
final_data.head()

In [ ]:
id_reg = final_data.drop_duplicates(['id_y','region'])[['id_y','region']].dropna().reset_index(drop=True)

In [ ]:
final_data[final_data['id_y']=='20'][['X', 'Y','region','Name']].value_counts()

In [ ]:
num_checkins_by_name = final_data[['id_x','id_y','Name','region']].groupby(['Name', 'id_y']).size().reset_index(name = "foot_fall")

In [ ]:
num_checkins_by_name = pd.merge(left=num_checkins_by_name, right=id_reg, how='left', on=['id_y'])

In [ ]:
num_checkins_by_name = num_checkins_by_name.rename(columns={'id_y': 'ID'})

In [ ]:
num_checkins_by_name.to_csv('../../data/sorted_data/footfall_per_region_sun.csv')

In [ ]:
final_data = final_data[final_data['type']=='check-in']

In [ ]:
num_checkins_by_name = final_data[['id_x','id_y','Name','region']].groupby(['Name', 'id_y']).size().reset_index(name = "checkins")
num_checkins_by_name = pd.merge(left=num_checkins_by_name, right=id_reg, how='left', on=['id_y'])
num_checkins_by_name = num_checkins_by_name.rename(columns={'id_y': 'ID'})

In [ ]:
num_checkins_by_name.to_csv('../../data/sorted_data/checkin_per_region_sun.csv')

In [ ]:
final_data = final_data.drop_duplicates(['id_x','X', 'Y'])
final_data = final_data.reset_index(drop=True)

In [ ]:
num_checkins_by_name = final_data[['id_x','id_y','Name','region']].groupby(['Name', 'id_y']).size().reset_index(name = "unique_checkins")
num_checkins_by_name = pd.merge(left=num_checkins_by_name, right=id_reg, how='left', on=['id_y'])
num_checkins_by_name = num_checkins_by_name.rename(columns={'id_y': 'ID'})

In [ ]:
num_checkins_by_name.to_csv('../../data/sorted_data/unique_checkin_per_region_sun.csv')

In [ ]:
final_data

In [ ]:
num_checkins_by_name